# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [ ]:
! pip install -U langchain

## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [18]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [19]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""


biology_template = """你是一位卓越的生物学家，精通生命的奥秘。
你擅长用简明扼要的方式解答各类生物学问题，从微观到宏观无所不能。
即使遇到未知问题，你也会诚实地承认，并积极寻找答案。

现在来看一个问题：
{input}"""


computer_template = """你是一位杰出的计算机科学家，驾驭着信息时代的浪潮。
你善于解决各种复杂的计算机问题，能够将问题分解成组成部分，逐一击破。
当面对未知时，你敢于承认不知道，并乐于进行深入探索。

让我们来看一个问题：
{input}"""


chinese_literature_template = """你是一位汉语言文学的权威，深谙语言之美。
你以清晰简洁的方式解答各类汉语言文学问题，如诗词、文言文等。
遇到不明了的问题，你会真诚承认，并愿意深入挖掘知识的深度。

让我们共同探讨一个问题：
{input}"""

In [20]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    {
        "name": "生物",
        "description": "适用于回答生物问题",
        "prompt_template": biology_template,
    },
    {
        "name": "汉语言文学",
        "description": "适用于回答汉语言文学问题",
        "prompt_template": chinese_literature_template,
    },
    {
        "name": "计算机",
        "description": "适用于回答计算机问题",
        "prompt_template": computer_template,
    }
]

In [21]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [22]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

In [23]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [24]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [25]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [26]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物问题', '汉语言文学: 适用于回答汉语言文学问题', '计算机: 适用于回答计算机问题']


In [27]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
汉语言文学: 适用于回答汉语言文学问题
计算机: 适用于回答计算机问题


In [28]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [29]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
汉语言文学: 适用于回答汉语言文学问题
计算机: 适用于回答计算机问题

<< INPUT >>
{input}

<

In [30]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [31]:
print(chain.invoke("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': '黑体辐射是什么？?'}
> Finished chain.
{'input': '黑体辐射是什么？?', 'text': '\n\n黑体辐射是指一个理想化的物体，在恒定温度下会发出的电磁辐射。这种辐射的特点是它的频率和温度有关，随着温度的升高，辐射的频率也会增加。此外，黑体辐射也具有良好的吸收和辐射能力，因此被广泛应用于光学和热学领域。'}


In [32]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': 'first prime number greater than 40 that when added by one is divisible by 3'}
> Finished chain.
{'input': 'first prime number greater than 40 that when added by one is divisible by 3', 'text': '\n\n首先，我们可以将问题分解为两个部分：\n1. 寻找大于40的第一个质数\n2. 将这个质数加一后，找到能够整除3的最小数\n\n第一部分，我们知道41是大于40的第一个质数，因此我们已经解决了第一个部分。\n\n第二部分，我们可以通过数学推理来解决。因为我们要找的数加一后能够整除3，\n所以这个数必须是3的倍数减一。因此，我们可以将41减一得到40，\n然后再找到最小的能够整除3的数，即37。\n\n因此，答案是37。 '}


In [33]:
router_chain.verbose = True

In [34]:
print(chain.invoke("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '什么是黑洞？'}
> Finished chain.
{'input': '什么是黑洞？', 'text': '\n\n黑洞是一种极度密集的天体，它拥有如此强大的引力，以至于连光都无法逃脱它的吸引力。它的存在是由于恒星的质量过大，导致它在死亡后无法继续支撑自身的引力，从而塌缩成一个极小的点，也被称为奇点。黑洞是宇宙中最奇特的物体之一，它们的存在依然是一个谜。但我们可以通过观测黑洞周围的物质和影响来推断它们的存在。黑洞有各种不同的种类，包括超大质量黑洞和中等质量黑洞。它们在宇宙中发挥着重要的作用，如维持星系的稳定和产生强烈的引力'}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains

In [35]:
print(chain.invoke("解释一下DNA和RNA的区别是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
生物: {'input': '解释一下DNA和RNA的区别是什么？'}
> Finished chain.
{'input': '解释一下DNA和RNA的区别是什么？', 'text': '\n\nDNA和RNA是两种不同的核酸分子，在生物体内起着重要的遗传作用。它们的区别主要体现在结构、功能和存在位置上。\n\n1. 结构：DNA是双链的螺旋结构，由四种碱基（腺嘌呤、鸟嘌呤、胸腺嘧啶和鸟嘌呤）组成，通过氢键连接成对，形成一个螺旋梯状的结构；而RNA是单链的，由三种碱基（腺嘌呤、鸟嘌呤、胸腺嘧啶和尿嘧啶）组成，没有双螺旋结构，但也可以形成部分二级结构。\n\n2. 功能：DNA是储存遗传信息的分子，它通过碱基序列来编码生物体的'}


In [36]:
print(chain.invoke("OSI七层分别是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
计算机: {'input': 'OSI七层分别是什么？'}
> Finished chain.
{'input': 'OSI七层分别是什么？', 'text': '\n\nOSI七层是指开放式系统互联参考模型（Open Systems Interconnection Reference Model）的七层结构，包括物理层、数据链路层、网络层、传输层、会话层、表示层和应用层。它是一个标准化的网络架构模型，用于描述计算机网络中数据的传输过程。每一层都有自己的功能和作用，它们依次由下至上为：\n\n1.物理层（Physical Layer）：负责传输物理数据信号，控制数据传输的电气和机械特性。\n\n2.数据链路层（Data Link Layer）：负责建立和维护两个相邻节点之间的数据链路，实现可靠的数据传输。\n\n3.网络层（Network Layer）：负责将数据包从源节点传输到目的节点，实现不同网络之间的通信。\n\n4.'}


In [37]:
print(chain.invoke("评价下诗人李白的作品风格？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
汉语言文学: {'input': '评价下诗人李白的作品风格？'}
> Finished chain.
{'input': '评价下诗人李白的作品风格？', 'text': '\n\n李白是中国古代最具有代表性的浪漫主义诗人，他的作品风格充满着奔放、豪迈和浪漫的气息。他的诗作大多描写自然景观，抒发个人感情，寄托理想抱负，带有强烈的感情色彩和想象力。\n\n首先，李白的诗歌语言具有华丽的艺术表现力，他善于运用比喻、夸张和象征等修辞手法，使诗作充满生动的意象和感染力。例如《将进酒》中的“红酥手，黄藤酒，满城春色宫墙柳”，就是典型的夸张手法，极具诗意。\n\n其次，李白的诗歌'}
